In [1]:

# %pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [3]:
# !gsutil cp /home/clas_giulia_s/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/clas_giulia_s/datasets/

In [4]:
# base_path = '/home/clas_giulia_s/buckets/b1/'
base_path = '/Users/ignacio/MAESTRIA/DMEF/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
# dataset_file = 'competencia_02_fe_v01.parquet'
dataset_file = 'competencia_01_fe_modelito.csv'

semillas = [261823, 289871, 379817, 481301, 959723]

# data = pd.read_parquet('/home/clas_giulia_s/datasets/' + dataset_file)
data = pd.read_csv(dataset_path + dataset_file)

In [5]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [6]:
# meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
#                202001, 202002, 202003, 202004, 202005, 202006,
#                202007, 202008, 202009, 202010, 202011, 202012,
#                202101, 202102, 202103, 202104, 202105, 202106]

meses_train = [202101, 202102, 202103, 202104]

data = data[data['foto_mes'].isin(meses_train)]

In [7]:
data.shape

(652447, 679)

In [8]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,202101,0.995180,0.004820
1,202102,0.993759,0.006241
2,202103,0.994037,0.005963
3,202104,0.992754,0.007246


In [9]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

Original proportion:0.00482021403972202
Original proportion:0.006240546954736053
Original proportion:0.005962672205761066
Original proportion:0.007246023523676031


In [10]:
data.shape

(69204, 679)

In [11]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,202101,0.954013,0.045987
1,202102,0.941258,0.058742
2,202103,0.943727,0.056273
3,202104,0.932436,0.067564


In [12]:
# output_file = 'competencia_02_fe_v01_undersampled.parquet'

output_file = 'competencia_01_fe_modelito_undersampled.csv'

# data.to_parquet('/home/clas_giulia_s/datasets/' + output_file, index=False)
data.to_csv(dataset_path + output_file, index=False)

In [13]:
# !gsutil cp /home/clas_giulia_s/datasets/competencia_02_fe_v01_undersampled.parquet /home/clas_giulia_s/buckets/b1/datasets/